In [276]:
import pandas
import lightfm
from lightfm import LightFM
from lightfm import cross_validation
from scipy.sparse import coo_matrix
from collections import OrderedDict
from scipy.sparse import coo_matrix
import numpy as np

In [277]:
df = pandas.read_csv('write.csv')

In [278]:
topNames = df.groupby("actionName").size().sort_values(ascending=False)[0:50].keys()
df = df[df.actionName.isin(topNames)]
actionByUsers = df.groupby(["userName","actionName"]).size()
actionByUsers=actionByUsers.apply(lambda x:1)
actionByUsers=actionByUsers.unstack()
actionByUsers=actionByUsers.fillna(0.0)
mat=coo_matrix(actionByUsers.values,dtype='float32')
users=actionByUsers.index.tolist()
items=list(actionByUsers.columns.values)

In [279]:
tags = pandas.read_json(path_or_buf="C:\\Users\\Robert\\python\\tagiDlaWebnei.json", orient = 'records', dtype={"A":str, "B":list})
tags[tags.title!=items[14]]
sum = []
for i in list(tags.tags.values):
    sum = sum + i
tagsList=list(OrderedDict.fromkeys(sum))

In [280]:
tagsMatrix = coo_matrix((len(items),len(tagsList)),dtype='float32')

In [284]:
rowM  = []
colM  = []
dataM = []
#np.array([])
for index, row in tags.iterrows():
    for rowA in row[1]:
        if((row[2] in items)&(rowA in tagsList)):
            rowM.append(items.index(row[2]))
            colM.append(tagsList.index(rowA))
            dataM.append(1.0)
rowM = np.array(rowM, dtype='float32')
colM  = np.array(colM, dtype='float32')
dataM = np.array(dataM, dtype='float32')
tagsMatrix = coo_matrix((dataM,(rowM,colM)),shape=(len(items),len(tagsList)))
tagsMatrix

<50x32 sparse matrix of type '<class 'numpy.float32'>'
	with 25 stored elements in COOrdinate format>

In [294]:
train, test = cross_validation.random_train_test_split(mat)
NUM_THREADS = 2
NUM_COMPONENTS = 30
NUM_EPOCHS = 3
ITEM_ALPHA = 1e-6

# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model = model.fit(train,item_features = tagsMatrix,epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train ,item_features = tagsMatrix, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Wall time: 361 ms
Collaborative filtering train AUC: 0.38730213


In [292]:
test_auc = auc_score(model,
                    test,
                    train_interactions=train,
                    item_features=tagsMatrix,
                    num_threads=NUM_THREADS).mean()
print('Hybrid test set AUC: %s' % test_auc)

Hybrid test set AUC: 0.28911808
